In [223]:
%load_ext autoreload
%autoreload 2

import os
import re
import dask.dataframe as dd
import pandas as pd
import numpy as np
import csv
from newspaper import Article
from articleDateExtractor import extractArticlePublishedDate
from dask import dataframe as dd
from dask.multiprocessing import get
from multiprocessing import cpu_count
from bs4 import BeautifulSoup

import plotly
import plotly.plotly as py
import plotly.graph_objs as go
plotly.offline.init_notebook_mode(connected=True)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [58]:
directory_con = 'data/conventional_data_pulls/'
directory_custom = 'data/custom_data_pulls/'

In [59]:
def concatenate_dataframes(pull_type, directory):
    """
    Concatenate all csv's and delete records with duplicate url's
    """
    # Year_month_day:
    pattern_date = '^\d{4}_\d{1,2}_\d{1,2}'

    # Short source name:
    pattern_source = '[a-z]+'
    
    # Starts with lower-case word and followed by letters/numbers/underscores (long source name):
    pattern_source_detail = '[a-z]+\w+'
    
    # Empty DataFrame to accumulate all csv's:
    df_master = pd.DataFrame()
    for filename in os.listdir(directory):
        # Skip non-csv's:
        if filename.endswith(".csv"):
            # Parse out date:
            pull_date = re.search(pattern_date, filename).group()
            pull_date = pd.to_datetime(pull_date, format='%Y_%m_%d')
            
            # Parse out short source name, e.g. 'rt':
            source = re.search(pattern_source, filename).group()
            if source == 'foxnews':
                source = 'fox'

            # And long source name, e.g. 'rt_main_page':
            source_detail = re.search(pattern_source_detail, filename).group()
            
            current_df = pd.read_csv(directory + filename)

            # Push pull_date and source name fields into the dataframe:
            current_df['source'] = source
            current_df['source_detail'] = source_detail
            current_df['pull_type'] = pull_type
            current_df['pull_date'] = pull_date

            # Accumulate frame:
            df_master = df_master.append(current_df, ignore_index=True)
            
            # Delete non-English RT sources (for conventional pulls only):
#             rt_filter = (df_master['source_url'] != 'http://actualidad.rt.com') & \
#                         (df_master['source_url'] != 'http://arabic.rt.com') & \
#                         (df_master['source_url'] != 'http://deutsch.rt.com') & \
#                         (df_master['source_url'] != 'http://francais.rt.com') & \
#                         (df_master['source_url'] != 'http://russian.rt.com') & \
#                         (df_master['source_url'] != 'http://rtd.rt.com')  # Exclude RT's documentary videos
#                         (df_master['source_url'] != 'http://rtd.rt.com')  # Exclude RT's documentary videos
#             df_master = df_master[rt_filter]

            # Clean up duplicates using 'canonical_link' and 'url':
            df_master.drop_duplicates(subset=['canonical_link', 'url'], keep='first', inplace=True)
    return df_master

In [44]:
# %%time
# df_conventional = concatenate_dataframes(pull_type='conventional', directory=directory_con)

CPU times: user 1min 19s, sys: 20.5 s, total: 1min 39s
Wall time: 1min 42s


In [90]:
%%time
df_custom = concatenate_dataframes(pull_type='custom', directory=directory_custom)

CPU times: user 1min 7s, sys: 13.6 s, total: 1min 20s
Wall time: 1min 22s


In [126]:
#df_custom.groupby('source').count()

,source_url,url,title,movies,text,keywords,meta_keywords,tags,authors,publish_date,...,rss_title,rss_link,rss_id,rss_published,rss_published_parsed,rss_feedburner_origlink,paper_section_name,source_detail,pull_type,pull_date
source,,,,,,,,,,,,,,,,,,,,,
bbc,797,797,797,797,797,797,797,797,797,0,...,797,797,797,797,797,797,797,797,797,797
cnn,939,939,939,939,939,939,939,939,939,0,...,939,939,939,939,939,939,939,939,939,939
fox,760,760,760,760,760,760,760,760,760,0,...,760,760,760,760,760,760,760,760,760,760
rt,831,831,831,831,831,831,831,831,831,0,...,831,831,831,831,831,831,831,831,831,831


In [93]:
#df_custom.groupby('source').count()

,source_url,url,title,movies,text,keywords,meta_keywords,tags,authors,publish_date,...,rss_title,rss_link,rss_id,rss_published,rss_published_parsed,rss_feedburner_origlink,paper_section_name,source_detail,pull_type,pull_date
source,,,,,,,,,,,,,,,,,,,,,
bbc,796,796,796,796,796,796,796,796,796,0,...,796,796,796,796,796,796,796,796,796,796
cnn,929,929,929,929,929,929,929,929,929,0,...,929,929,929,929,929,929,929,929,929,929
fox,760,760,760,760,760,760,760,760,760,0,...,760,760,760,760,760,760,760,760,760,760
rt,1,1,1,1,1,1,1,1,1,0,...,1,1,1,1,1,1,1,1,1,1


In [46]:
#df_master = pd.concat([df_conventional, df_custom], ignore_index=True)

In [48]:
#df_master.loc[3048, 'canonical_link'] = df_master.loc[3048, 'url']

In [49]:
#df_master['meta_data'] = df_master['meta_data'].astype(str)

### Work on filling RT publish dates:

In [61]:
def extract_rt_html_date(html_str):
    """
    Extracts based on html tag
    """
    soup = BeautifulSoup(html_str, 'html.parser')
    for item in soup.find_all('meta'):
        if item.get('name') == 'published_time_telegram':
            date_str = item.get('content', np.NaN)
            return pd.to_datetime(date_str)
    return np.nan

# def extract_rt_html_date_2(html_str):
#     """
#     """
#     soup = BeautifulSoup(html_str, 'html.parser')
#     for item in soup.find_all('span'):
#         if item.get('name') == 'published_time_telegram':
#             date_str = item.get('content', 'NotFound')
#             return pd.to_datetime(date_str)
#     return np.nan

def extract_rt_html_date_2(html_str):
    """
    Extracts based on text
    """
    regex = '[0-9]+\s[A-Za-z]+,\s[0-9]+'
    soup = BeautifulSoup(html_str, 'html.parser')
    for item in soup.find_all('span'):
        find_class = item.get('class', np.NaN)
        if find_class == ['date', 'date_article-header']:
            date_str = item.get_text()
            date_str = re.search(regex, date_str).group()
            return pd.to_datetime(date_str)
    return np.NaN

First pass:

In [91]:
rt_df = df_custom[df_custom.source == 'rt']

In [92]:
%%time
results_rt = rt_df['html'].apply(extract_rt_html_date)

CPU times: user 33.8 s, sys: 587 ms, total: 34.3 s
Wall time: 34.5 s


In [116]:
# Update the dataframe with discovered publish dates:
df_custom['publish_date'] = df_custom['publish_date'].fillna(results_rt)

In [182]:
results_rt.notnull().sum()

743

In [119]:
df_custom['publish_date'].isnull().sum()

2584

In [120]:
df_custom.shape

(3327, 27)

In [123]:
df_custom.groupby('source').count()

,source_url,url,title,movies,text,keywords,meta_keywords,tags,authors,publish_date,...,rss_title,rss_link,rss_id,rss_published,rss_published_parsed,rss_feedburner_origlink,paper_section_name,source_detail,pull_type,pull_date
source,,,,,,,,,,,,,,,,,,,,,
bbc,797,797,797,797,797,797,797,797,797,0,...,797,797,797,797,797,797,797,797,797,797
cnn,939,939,939,939,939,939,939,939,939,0,...,939,939,939,939,939,939,939,939,939,939
fox,760,760,760,760,760,760,760,760,760,0,...,760,760,760,760,760,760,760,760,760,760
rt,831,831,831,831,831,831,831,831,831,743,...,831,831,831,831,831,831,831,831,831,831


In [99]:
#df_master.loc[:, ('canonical_link', 'publish_date')][df_master.publish_date.notnull()].loc[2452].values

Second pass:

In [124]:
rest_rt_df = df_custom[(df_custom.source == 'rt') & (df_custom.publish_date.isnull())]

In [125]:
rest_rt_df.shape

(88, 27)

In [126]:
%%time
results_rest_rt = rest_rt_df['html'].apply(extract_rt_html_date_2)

CPU times: user 1.97 s, sys: 12 ms, total: 1.98 s
Wall time: 1.98 s


In [185]:
results_rest_rt.notnull().shape

(88,)

In [127]:
# Push the rest of the discovered dates in:
df_custom['publish_date'] = df_custom['publish_date'].fillna(results_rest_rt)

In [128]:
df_custom.publish_date.isnull().sum()

2496

### Work on filling out non-RT publish dates

In [143]:
def error_wrapper_date_parser(url):
    """
    """
    try:
        return extractArticlePublishedDate(url)
    except:
        return np.NaN

In [130]:
non_rt_df = df_custom[df_custom.source != 'rt']

In [131]:
nCores = cpu_count()
print('CPU Cores:', nCores)

CPU Cores: 8


In [16]:
%%time
dask_non_rt_df = dd.from_pandas(non_rt_df, npartitions=nCores)

CPU times: user 7.12 s, sys: 10.4 s, total: 17.5 s
Wall time: 29.7 s


In [145]:
%%time
results_non_rt = dask_non_rt_df['url'].map_partitions(lambda ser : ser.apply(error_wrapper_date_parser)).compute(scheduler='processes')

Extracting date from foo
Exception in extractArticlePublishedDate for foo
("unknown url type: 'foo'",)
Extracting date from foo
Exception in extractArticlePublishedDate for foo
("unknown url type: 'foo'",)
Extracting date from https://www.bbc.co.uk/news/business-47063405
Extracting date from https://www.bbc.co.uk/news/business-47066873
Extracting date from https://www.bbc.co.uk/news/business-47057870
Extracting date from https://www.cnn.com/2019/01/30/politics/shutdown-republicans-poll/index.html
Extracting date from https://www.bbc.co.uk/news/business-47055188
Extracting date from https://www.cnn.com/2019/01/30/politics/trump-foxconn-manufacturing/index.html
Extracting date from https://www.cnn.com/2019/01/30/politics/sarah-sanders-god-trump/index.html
Extracting date from https://www.cnn.com/2019/01/30/politics/donald-trump-reelection-numbers-2020/index.html
Extracting date from https://www.cnn.com/2019/01/30/politics/undocumented-worker-trump-state-of-the-union/index.html
Extracting

Extracting date from https://www.cnn.com/2019/01/30/tech/new-york-attorney-general-social-media/index.html
Extracting date from https://www.foxnews.com/world/japanese-police-launch-probe-into-an-online-sellers-effort-to-sell-apparent-uranium
Extracting date from https://www.foxnews.com/sports/super-bowl-liii-rams-vs-patriots
Extracting date from https://www.cnn.com/2019/01/30/tech/dhs-h1b-final-rule/index.html
Extracting date from https://www.bbc.co.uk/news/stories-47093041
Extracting date from https://www.foxnews.com/us/missing-kentucky-mom-taken-to-rural-home-police-say
Extracting date from https://www.foxnews.com/world/at-least-60-dead-in-boko-haram-attack-report
Extracting date from https://www.foxnews.com/us/border-patrol-agent-killed-while-on-duty-in-texas-officials
Extracting date from https://www.foxnews.com/politics/trump-picks-patriots-to-win-super-bowl-liii-while-poll-finds-half-a-country-doesnt-care-who-wins
Extracting date from https://www.cnn.com/2019/01/29/tech/facetime-

Extracting date from https://www.bbc.co.uk/news/world-us-canada-47143288
Extracting date from https://www.bbc.co.uk/news/entertainment-arts-47149342
Extracting date from https://www.cnn.com/2019/01/30/us/el-chapo-guzman-trial-closing-arguments/index.html
Extracting date from https://www.foxnews.com/tech/facebook-removes-accounts-of-fake-news-group-in-indonesia
Extracting date from https://www.bbc.co.uk/news/business-47054322
Extracting date from https://www.foxnews.com/tech/how-to-get-the-best-deals-on-macbook-ipad-and-iphone
Extracting date from https://www.bbc.co.uk/news/uk-england-humber-47102169
Extracting date from https://www.cnn.com/2019/01/30/sport/roger-goodell-super-bowl-liii-press-conference/index.html
Extracting date from https://www.foxnews.com/tech/from-apple-tv-to-roku-5-streaming-tv-devices-compared
Extracting date from https://www.bbc.co.uk/sport/football/46996820
Extracting date from https://www.foxnews.com/us/cow-shot-after-charging-police-officer-following-escape-fr

Extracting date from https://www.cnn.com/travel/article/shanghai-disneyland-zootopia/index.html
Extracting date from https://www.bbc.co.uk/news/uk-wales-47072503
Extracting date from https://www.cnn.com/travel/article/banana-island-qatar/index.html
Extracting date from https://www.bbc.co.uk/news/uk-politics-47143135
Extracting date from https://www.cnn.com/travel/article/japan-winter-stove-train-tsugaru/index.html
Extracting date from https://www.bbc.co.uk/news/entertainment-arts-47133868
Extracting date from https://www.bbc.co.uk/news/entertainment-arts-47150188
Extracting date from https://www.bbc.co.uk/news/world-latin-america-47101291
Extracting date from https://www.cnn.com/travel/article/igloo-hotel-davos-switzerland/index.html
Extracting date from https://www.bbc.co.uk/news/business-47124328
Extracting date from https://www.bbc.co.uk/news/business-46924999
Extracting date from https://www.cnn.com/travel/article/italy-town-pays-people-to-move-intl/index.html
Extracting date from 

Extracting date from https://www.bbc.co.uk/news/health-47150658
Extracting date from https://www.cnn.com/2019/01/29/europe/unhcr-mediterranean-migrant-deaths-intl/index.html
Extracting date from https://www.bbc.co.uk/news/newsbeat-47127298
Extracting date from https://www.bbc.co.uk/news/business-46985443
Extracting date from https://www.bbc.co.uk/news/technology-47069414
Extracting date from https://www.cnn.com/2019/01/29/asia/india-nuns-kerala-intl/index.html
Extracting date from https://www.bbc.co.uk/news/world-latin-america-47104508
Extracting date from https://www.cnn.com/2019/01/15/tech/alibaba-city-brain-hangzhou/index.html
Extracting date from https://www.bbc.co.uk/news/entertainment-arts-47114899
Extracting date from https://www.cnn.com/2019/01/26/intl_business/davos-divide/index.html
Extracting date from https://www.bbc.co.uk/sport/rugby-union/46799568
Extracting date from https://www.bbc.co.uk/news/business-47083972
Extracting date from https://www.bbc.co.uk/news/uk-england-w

...

Extracting date from https://www.foxnews.com/tech/army-tests-new-communication-device-that-could-save-wounded-soldiers-lives
Extracting date from https://www.bbc.co.uk/news/business-47155720
Extracting date from https://www.foxnews.com/us/alleged-nyc-subway-shooter-suspected-ms-13-member-was-in-us-illegally-ice-says
Extracting date from https://www.foxnews.com/us/virginia-state-trooper-dies-after-suspect-opened-fire-during-raid
Extracting date from https://www.cnn.com/us/live-news/super-bowl-2019-updates/index.html
Extracting date from https://www.foxnews.com/us/supreme-court-halts-louisiana-abortion-clinic-law-for-now
Extracting date from https://www.foxnews.com/us/fox-news-first-trump-slams-bad-for-country-pelosi-northam-under-pressure-to-resign-over-racist-pic
Extracting date from https://www.foxnews.com/us/trump-to-call-for-unity-at-state-of-the-union-while-dems-push-disunity-dems-dilemma-in-northam-scandal
Extracting date from https://www.foxnews.com/us/texas-man-37-gets-60-years-

Extracting date from https://www.foxnews.com/world/paris-building-fire-death-toll-rises-to-10-arson-eyed
Extracting date from https://www.bbc.co.uk/news/stories-46964969
Extracting date from https://www.cnn.com/2019/02/03/cnn10/ten-content-mon/index.html
Extracting date from https://www.foxnews.com/world/russia-must-develop-new-missile-systems-in-response-to-us-withdrawal-from-arms-control-treaty-russia-top-minister-says
Extracting date from https://www.foxnews.com/world/pope-francis-celebrates-first-ever-papal-mass-in-united-arab-emirates
Extracting date from https://www.bbc.co.uk/news/business-47157411
Extracting date from https://www.cnn.com/2019/02/03/opinions/presidential-weekly-briefing-sotu-threats-vinograd/index.html
Extracting date from https://www.bbc.co.uk/news/science-environment-47030057
Extracting date from https://www.bbc.co.uk/news/business-47144762
Extracting date from https://www.cnn.com/2019/02/03/football/premier-league-arsenal-manchester-city-manchester-united-leic

Extracting date from https://www.bbc.co.uk/news/entertainment-arts-47147570
Extracting date from https://www.bbc.co.uk/news/science-environment-46960842
Extracting date from https://www.bbc.co.uk/news/entertainment-arts-47144368
Extracting date from https://www.bbc.co.uk/news/world-asia-46943593
Extracting date from https://www.wkbw.com/news/local-news/amherst-and-buffalo-police-officers-honored-for-lifting-vehicle-freeing-woman-trapped-beneath
Extracting date from https://www.fox4now.com/news/local-news/punta-gorda-day-care-worker-charged-with-child-abuse
Extracting date from https://www.3newsnow.com/news/local-news/nsp-seizes-189-pounds-of-marijuana-during-i-80-stop
Extracting date from https://www.wnem.com/news/baby-born-at-saginaw-homeless-shelter/article_7115e778-2694-11e9-af52-475ed88778f2.html
Extracting date from https://www.bbc.co.uk/news/entertainment-arts-47143399
Extracting date from https://www.bbc.co.uk/news/science-environment-46951152
Extracting date from https://www.kt

Extracting date from https://www.cnn.com/2019/01/30/entertainment/ariana-grande-tattoo/index.html
Extracting date from https://www.cnn.com/2019/01/30/entertainment/maroon-5-travis-scott-big-boi-halftime-performance-what-to-know/index.html
Extracting date from https://www.cnn.com/2019/01/29/entertainment/reactions-jussie-smollett/index.html
Extracting date from https://www.cnn.com/2019/01/30/entertainment/celine-dion-thin/index.html
Extracting date from https://www.cnn.com/2019/01/30/entertainment/gwyneth-paltrow-lawsuit-skiing/index.html
Extracting date from https://www.cnn.com/2019/01/29/entertainment/netflix-hulu-amazon-february/index.html
Extracting date from https://www.cnn.com/2019/01/30/entertainment/nick-cannon-wendy-williams-show/index.html
Extracting date from https://www.cnn.com/2019/01/29/entertainment/netflix-ted-bundy/index.html
Extracting date from https://www.cnn.com/2019/01/30/entertainment/bts-v-song-scenery/index.html
Extracting date from https://www.cnn.com/2019/01/2

Extracting date from https://www.cnn.com/2019/01/30/politics/homeland-security-chairman-subpoena-nielsen/index.html
CPU times: user 6.97 s, sys: 16 s, total: 23 s
Wall time: 11min 30s


In [189]:
results_non_rt.notnull().sum()

2419

In [165]:
# Some time standards conversion:
results_non_rt_converted = pd.to_datetime(results_non_rt, utc=True)
results_non_rt_converted = results_non_rt_converted.dt.tz_localize(None)

In [191]:
df_custom['publish_date'] = df_custom['publish_date'].fillna(results_non_rt_converted)

In [196]:
# 77 makes sense
df_custom['publish_date'].isnull().sum()

77

In [214]:
df_custom.groupby('source').count()

,source_url,url,title,movies,text,keywords,meta_keywords,tags,authors,publish_date,...,rss_title,rss_link,rss_id,rss_published,rss_published_parsed,rss_feedburner_origlink,paper_section_name,source_detail,pull_type,pull_date
source,,,,,,,,,,,,,,,,,,,,,
bbc,797,797,797,797,797,797,797,797,797,795,...,797,797,797,797,797,797,797,797,797,797
cnn,939,939,939,939,939,939,939,939,939,928,...,939,939,939,939,939,939,939,939,939,939
fox,760,760,760,760,760,760,760,760,760,696,...,760,760,760,760,760,760,760,760,760,760
rt,831,831,831,831,831,831,831,831,831,831,...,831,831,831,831,831,831,831,831,831,831


In [234]:
%%time
# Back_up
df_custom.to_csv('data/2019_02_09_analysis.csv')

CPU times: user 47.3 s, sys: 10.3 s, total: 57.7 s
Wall time: 1min 9s


In [230]:
data = [go.Histogram(x=df_custom.publish_date)]

In [231]:
plotly.offline.iplot({
    "data": data,
    "layout": go.Layout(title="hello world")
})

### Analysis

In [207]:
test_url = 'https://www.bbc.co.uk/news/uk-47063009'
test_url = 'https://www.rt.com/news/450875-china-nuclear-aircraft-carriers/'
test_url = 'https://www.rt.com/usa/450846-nbc-rt-hunt-tulsi/'
test_url = 'https://www.rt.com/business/450243-iran-eu-transaction-channel/'
test_url = 'https://www.cnn.com/2018/10/03/media/next-popular-tv-show/index.html'
test_url = 'https://www.nbcnews.com/politics/congress/bipartisan-venezuela-legislation-fizzles-over-use-military-force-n968581'
test_url = 'https://www.nbcnews.com/business/economy/trump-touts-thriving-steel-industry-manufacturing-insiders-disagree-n954866'
test_url = 'https://www.rt.com/news/450205-castles-disney-turkey-video/'
test_url = 'https://www.rt.com/news/450329-usa-inf-treaty-russia-arms/'
extractArticlePublishedDate(test_url)

Extracting date from https://www.rt.com/news/450329-usa-inf-treaty-russia-arms/


datetime.datetime(2019, 2, 8, 6, 18, tzinfo=tzutc())

In [180]:
df_master[(df_master.source == 'rt') & (df_master.rss_published_parsed.notnull())]#.publish_date

,article_html,authors,canonical_link,html,keywords,meta_data,meta_description,meta_keywords,movies,paper_section_name,...,rss_published_parsed,rss_title,source,source_detail,source_url,summary,tags,text,title,url
3048,NaN,[],https://www.rt.com/news/450205-castles-disney-...,"<!DOCTYPE html>\n<html prefix=""og: http://ogp....",[],"defaultdict(<type 'dict'>, {'description': 'Hu...","Hundreds of chateaux, seemingly plucked out of...",[''],[],rt_main_page,...,"time.struct_time(tm_year=2019, tm_mon=1, tm_md...",Eerie DRONE VIDEO shows 580+ abandoned Disney-...,rt,rt_main_page,https://www.rt.com/,NaN,set([u'Turkey news']),"Hundreds of chateaux, seemingly plucked out of...",Eerie DRONE VIDEO shows 580+ abandoned Disney-...,https://www.rt.com/news/450205-castles-disney-...


In [129]:
df_master[df_master.canonical_link.isnull()]['canonical_link']

3048    NaN
Name: canonical_link, dtype: object

### Force download and parsing of articles that we failed to capture

In [39]:
df2.loc[838, 'url']

'https://www.cnn.com/2017/07/24/asia/return-to-cambodia-sex-trafficking/index.html'

In [41]:
article = Article(df2.loc[838, 'url'])

In [47]:
article.download()

In [50]:
article.parse()

In [51]:
article.text

'Svay Pak, Cambodia (CNN) Sephak was aged just 13 years old when she was sold for sex by her mother.\n\nShe was taken to a hospital, issued a certificate confirming her virginity, and then taken to a hotel room where a she was raped for days. She was returned home after three nights.\n\nSephak grew up in Svay Pak, a poor fishing village on the outskirts of the Cambodian capital of Phnom Penh. It\'s a community that has become notorious as a place to buy child sex.\n\nHer mother, Ann, said her family had fallen on hard times and that they took out a loan that eventually spiraled to about $6,000 in debt. With money-lenders threatening her, Ann took up an offer from a woman who approached her promising big money for her daughter\'s virginity.\n\nSephak said her mother was paid $800. But after Sephak\'s return, her mother began pressuring her to work in a brothel.\n\nAnn said she regrets her decision and that if she had known then what she knows now, she would never have sold her daughter.